In [15]:
#!pip install tabpfn

In [16]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

data = pd.read_csv('DATASETS/Project_dataset.csv')

In [17]:
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_and_song,genre,Lyrics
0,0.03100,0.610,200400,0.926,0.001200,0.000000,0.0821,-4.843,0.0,0.0479,172.638,0.0,0.861,*nsync /// bye bye bye,"['Dance', 'Pop']","(Hey, Hey)\nBye, Bye, Bye\nBye, Bye\nBye, Bye\..."
1,0.08660,0.748,253600,0.811,0.000015,0.083333,0.0282,-5.213,0.0,0.0907,93.963,0.0,0.862,*nsync /// girlfriend,['Dance'],Would you be my girlfriend?\nWould you be my g...
2,0.43000,0.704,292000,0.409,0.000000,0.166667,0.1090,-8.581,0.0,0.0597,113.863,0.0,0.495,*nsync /// gone,['Dance'],"There's a thousand words that I could say,\nTo..."
3,0.00993,0.810,206333,0.857,0.000011,0.250000,0.0528,-4.658,0.0,0.0881,104.502,0.0,0.646,*nsync /// it makes me ill,['Dance'],Justin:\nI was hanging with the fellas\nSaw yo...
4,0.04070,0.636,192427,0.873,0.000001,0.083333,0.0372,-4.672,0.0,0.0710,165.071,0.0,0.908,*nsync /// it's gonna be me,"['Dance', 'Pop']",You might be hurt babe\nThat ain't no lie\nYou...


### Audio Baseline

In [18]:
from sklearn.preprocessing import LabelEncoder

# Load the data
labels = data["genre"].tolist()
#print(labels)
data_audio = data.drop(columns=['genre', 'artist_and_song', 'Lyrics'])

# Encode the labels
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_audio, labels, test_size=0.2)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


[156 196 196 ... 189 196 189]


In [19]:
# Initialize the model
clf = MLPClassifier(hidden_layer_sizes=(256,128,64), max_iter=200)

# Train the model
clf.fit(X_train, y_train)

# # Evaluate the model
# train_acc = clf.score(X_train, y_train)
# test_acc = clf.score(X_test, y_test)
# print(f"Train accuracy: {train_acc:.2f}, Test accuracy: {test_acc:.2f}")

y_pred = clf.predict(X_test)

# Calculate the classification report for this fold
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00        10
          15       0.09      0.08      0.09        12
          17       0.00      0.00      0.00         0
          18       0.00      0.00      0.00        20
          19       0.00      0.00      0.00         0
          20       0.18      0.24      0.21        25
          21       0.25    

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Pr